In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import spacy
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()# Visualise inside a notebook
import en_core_web_md
from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaMulticore
from gensim.models import CoherenceModel

/Users/bhaveshchainani/virtualenv/proofpoint_code/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/bhaveshchainani/virtualenv/proofpoint_code/lib/python3.8/site-packages/gensim/matutils.py:22: DeprecationWarning: Please use `triu` from the `scipy.linalg` namespace, the `scipy.linalg.special_matrices` namespace is deprecated.
  from scipy.linalg.special_matrices import triu


In [4]:
lang_interested = "en"

In [10]:
df_comb = pd.read_excel("../files/input/combined_data.xlsx")

In [11]:
df_comb

,person,content
0,George Clooney,"George Timothy Clooney (born May 6, 1961) is a..."
1,Shah Rukh Khan,Shah Rukh Khan (pronounced [ˈʃɑːɦɾʊx xɑːn]; bo...
2,Leonardo DiCaprio,Leonardo Wilhelm DiCaprio (; Italian: [diˈkaːp...
3,Will Smith,"Willard Carroll Smith II (born September 25, ..."
4,Kamal Haasan,Kamal Haasan (born 7 November 1954) is an Indi...
...,...,...
148,"Трамп, Дональд",До́нальд Джон Трамп (англ. Donald John Trump; ...
149,"Обама, Барак",Бара́к Хуссе́йн Оба́ма II (англ. Barack Hussei...
150,"Кэмерон, Дэвид","Дэвид Уильям Дональд Кэ́мерон (Камерон, англ. ..."
151,"Клинтон, Хиллари",Хи́ллари Дайа́н Ро́дэм Кли́нтон (англ. Hillary...


In [29]:
a = df_comb[df_comb["actual_lang"]=="en"]
a[a.duplicated('person')]

,person,content,actual_lang
55,Kamal Haasan,"Kamal Haasan (Ramanathapuram, 7 novembre 1954)...",en
78,James Patterson,"James Patterson (Newburgh, 22 marzo 1947) è un...",en


In [ ]:
## lang detect python

In [21]:
from langdetect import detect, DetectorFactory
DetectorFactory.seed = 0

df_comb["actual_lang"] = df_comb["content"].apply(detect)

In [22]:
df_comb.actual_lang.value_counts()

en    53
ru    51
it    49
Name: actual_lang, dtype: int64

In [16]:
df_comb[""]

,person,content,actual_lang
0,George Clooney,"George Timothy Clooney (born May 6, 1961) is a...",en
1,Shah Rukh Khan,Shah Rukh Khan (pronounced [ˈʃɑːɦɾʊx xɑːn]; bo...,en
2,Leonardo DiCaprio,Leonardo Wilhelm DiCaprio (; Italian: [diˈkaːp...,en
3,Will Smith,"Willard Carroll Smith II (born September 25, ...",en
4,Kamal Haasan,Kamal Haasan (born 7 November 1954) is an Indi...,en
...,...,...,...
148,"Трамп, Дональд",До́нальд Джон Трамп (англ. Donald John Trump; ...,ru
149,"Обама, Барак",Бара́к Хуссе́йн Оба́ма II (англ. Barack Hussei...,ru
150,"Кэмерон, Дэвид","Дэвид Уильям Дональд Кэ́мерон (Камерон, англ. ...",ru
151,"Клинтон, Хиллари",Хи́ллари Дайа́н Ро́дэм Кли́нтон (англ. Hillary...,ru


In [ ]:
## filter to required lang

In [ ]:
lang_models = {"en": spacy.load("en_core_web_md"), "it": spacy.load("it_core_news_md"), "ru": spacy.load("ru_core_news_md")}

In [ ]:
# Our spaCy model:
nlp = en_core_web_md.load()
# Tags I want to remove from the text
removal= ['ADV','PRON','CCONJ','PUNCT','PART','DET','ADP','SPACE', 'NUM', 'SYM']
tokens = []
for summary in nlp.pipe(reports['summary']):
    proj_tok = [token.lemma_.lower() for token in summary if token.pos_ not in removal and not token.is_stop and token.is_alpha]
    tokens.append(proj_tok)

In [ ]:
def format_topics_sentences(ldamodel=None, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data_ready)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
df_dominant_topic.head(10)